In [ ]:
!pip install fosforml
!pip install statsmodels

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [2]:
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()

In [3]:
df = 'ATTRITION_PREDICTIVE_DATA'

In [4]:
sf_df = my_session.sql("select * from {}".format(df))

In [5]:
import pandas as pd
pandas_df = sf_df.to_pandas()

In [6]:
print(pandas_df.isnull().sum())

EMPLOYEE_ID                      0
TENURE_MONTHS                    0
BIRTH_YEAR                       0
AGE                              0
SENIORITY                        0
SCHOOL_ENDDATE                   0
JOB_STARTDATE                    0
JOB_ENDDATE                  90076
CITY                             0
DISTANCE                         0
DEGREE_CLEAN                     0
ETHNICITY                        0
MARITAL_STATUS                   0
ROLE                             0
COMPANY_NAME                     0
ORGANIZATION_TYPE                0
ORGANIZATION_OWNERSHIP           0
STATE                            0
COUNTRY                          0
GENDER                           0
WORK_LIFE_BALANCE                0
BUSINESS_TRAVEL                  0
ENVIRONMENT_SATISFACTION         0
SALARY                           0
JOB_SATISFACTION                 0
PERCENTAGE_SALARY_HIKE           0
PERFORMANCE_RATING               0
OVER_TIME                        0
RELATIONSHIP_SATISFA

In [7]:
Original_df = pandas_df.dropna()

In [8]:
print(Original_df.isnull().sum())

EMPLOYEE_ID                  0
TENURE_MONTHS                0
BIRTH_YEAR                   0
AGE                          0
SENIORITY                    0
SCHOOL_ENDDATE               0
JOB_STARTDATE                0
JOB_ENDDATE                  0
CITY                         0
DISTANCE                     0
DEGREE_CLEAN                 0
ETHNICITY                    0
MARITAL_STATUS               0
ROLE                         0
COMPANY_NAME                 0
ORGANIZATION_TYPE            0
ORGANIZATION_OWNERSHIP       0
STATE                        0
COUNTRY                      0
GENDER                       0
WORK_LIFE_BALANCE            0
BUSINESS_TRAVEL              0
ENVIRONMENT_SATISFACTION     0
SALARY                       0
JOB_SATISFACTION             0
PERCENTAGE_SALARY_HIKE       0
PERFORMANCE_RATING           0
OVER_TIME                    0
RELATIONSHIP_SATISFACTION    0
CHURN                        0
dtype: int64


In [9]:
df = Original_df[["DISTANCE","DEGREE_CLEAN","ROLE","COMPANY_NAME","ORGANIZATION_TYPE","ORGANIZATION_OWNERSHIP","JOB_ENDDATE","CHURN"]]

In [10]:
df

,DISTANCE,DEGREE_CLEAN,ROLE,COMPANY_NAME,ORGANIZATION_TYPE,ORGANIZATION_OWNERSHIP,JOB_ENDDATE,CHURN
0,<2 miles,Bachelors Degree,Accounting Manager,Horizon Financials,Financial Services,Private,2019-08-01,1
1,<2 miles,Undergraduate Diploma,Compliance Manager,Vertex Dynamics,Aerospace,Private,2021-06-01,1
3,<2 miles,Bachelors Degree,Process Assistant,Horizon Financials,Financial Services,Private,2017-07-01,1
4,<2 miles,Bachelors Degree,Operations Manager,BrightStart Ventures,E-commerce,Private,2019-08-01,1
6,>10 miles,Other,Operations Manager,BrightStart Ventures,E-commerce,Private,2018-07-01,1
...,...,...,...,...,...,...,...,...
299995,>10 miles,Masters Degree,Corporate Counsel,BridgePoint Capital,Investment,Public,2023-05-01,1
299996,>10 miles,Masters Degree,Compliance Manager,SecureTech Solutions,Technology,Public,2023-05-01,1
299997,>10 miles,Masters Degree,Process Assistant,GreenWorld Foundation,Environmental,Non-Profit,2023-05-01,1
299998,>10 miles,Masters Degree,Data Scientist,Pinnacle Technologies,Technology,Public,2023-05-01,1


In [11]:
df['JOB_ENDDATE'] = pd.to_datetime(df['JOB_ENDDATE'])

In [12]:
df.set_index('JOB_ENDDATE', inplace = True)

In [13]:
# Group by categorical columns and resample monthly, then sum CHURN
New_df = df.groupby(["DISTANCE", "DEGREE_CLEAN", "ROLE", "COMPANY_NAME", "ORGANIZATION_TYPE", "ORGANIZATION_OWNERSHIP"]).resample('M').agg({'CHURN': 'sum'}).reset_index()

In [14]:
# Filter rows where CHURN is greater than 0
df1 = New_df[New_df['CHURN'] > 0]

In [15]:
df1

,DISTANCE,DEGREE_CLEAN,ROLE,COMPANY_NAME,ORGANIZATION_TYPE,ORGANIZATION_OWNERSHIP,JOB_ENDDATE,CHURN
0,2-5 miles,Associates Degree,Account Executive,AlphaRetail Ltd.,Retail,Public,2015-10-31,1
36,2-5 miles,Associates Degree,Account Executive,AlphaRetail Ltd.,Retail,Public,2018-10-31,1
43,2-5 miles,Associates Degree,Account Executive,AlphaRetail Ltd.,Retail,Public,2019-05-31,1
72,2-5 miles,Associates Degree,Account Executive,AlphaRetail Ltd.,Retail,Public,2021-10-31,1
74,2-5 miles,Associates Degree,Account Executive,AlphaRetail Ltd.,Retail,Public,2021-12-31,1
...,...,...,...,...,...,...,...,...
1569542,>10 miles,Undergraduate Diploma,Warehouse Associate,Streamline Networks,Telecommunications,Public,2014-10-31,1
1569543,>10 miles,Undergraduate Diploma,Warehouse Associate,TechNova Inc.,Technology,Public,2020-01-31,1
1569544,>10 miles,Undergraduate Diploma,Warehouse Associate,UrbanGreen Ltd.,Environmental Services,Private,2014-10-31,1
1569545,>10 miles,Undergraduate Diploma,Warehouse Associate,Vertex Dynamics,Aerospace,Private,2020-01-31,2


In [ ]:
# Split data into training and testing sets
train = df1[:'2024-08']
test = df1['2024-09':]

In [ ]:
df['JOB_ENDDATE'] = pd.to_datetime(df['JOB_ENDDATE'])
monthly_data = df.resample('M', on='JOB_ENDDATE').sum()

In [ ]:
monthly_data